In [1]:
import test_bench
import json
import pandas as pd
import importlib
importlib.reload(test_bench)

<module 'test_bench' from 'd:\\xcc\\大学\\保研\\电院\\mer_ov_exp1\\test_bench.py'>

In [2]:
prediction_emotion_strs = []
sample_ids = []
with open("result.json", "r", encoding="utf-8") as f:
    prediction_json = json.load(f)
for item in prediction_json["data"]:
    prediction_emotion_strs.append(item["emotion"])
    sample_ids.append(item["id"])

    df = pd.read_csv("final-openset-chinese.csv")
    true_emotion_strs = list(df.iloc[:, 1].to_numpy())



In [4]:
test_bench.test(prediction_emotion_strs, true_emotion_strs, sample_ids)

sample 0: accuracy: 0.0, recall: 0.0, score: 0.0


,sample_index,sample,emotion_group,accuracy,recall,score
0,0,sample_00000000,"[[激动], [痛苦, 难受, 伤心, 悲伤], [愉快, 轻松], [自嘲, 幽默]]",0.0,0.0,0.0
